# import libraries

In [1]:
import time
import face_recognition
from matplotlib import pyplot as plt
import cv2
from glob import glob as gl
import numpy as np
import tensorflow as tf
model_path = "export_models/RFB/"
model = tf.keras.models.load_model(model_path)

# Define some functions

In [6]:
def load_user(path):
    images = gl(path+"/*.jpg")
    info_file = path+"/info.txt"
    f = open(info_file, "r")
    content = f.readlines()
    info = {"uid": content[0][:-1],
             "name": content[1][:-1],
             "age": content[2][:-1],
             "permission": content[3][:-1],
             "more": content[4][:-1]
            }
    encoded_faces = []
    for image in images:
        img = face_recognition.load_image_file(image)
        h, w = img.shape[:2]
        encoded_face = face_recognition.face_encodings(img, known_face_locations=[(0, w, h, 0)])[0]
        encoded_faces.append(encoded_face)
        
    return {"info":info, "vectors": encoded_faces}
        
def load_data():
    paths = gl("users/*")
    data = {}
    for path in paths:
        user = load_user(path)
        data[user.get("info").get("uid")] = user
    return data

def distances_compute(face, user):
    h, w = face.shape[:2]
    encoded_face = face_recognition.face_encodings(face, known_face_locations=[(0, w, h, 0)])[0]
    vectors = user['vectors']
    
    distances = face_recognition.face_distance(vectors, encoded_face)
    
    return distances

def user_verify(face):
    min_distance = 2
    found_key = None
    user_matched = None
    for key in keys:
        distances = distances_compute(face, data[key])
        mean_dis, min_dis = np.mean(distances), np.min(distances)
        if min_dis < min_distance:
            found_key = key
            min_distance = min_dis
            user_matched = data[found_key]["info"]
    if min_distance<0.4:
        return min_distance, user_matched
    else:
        return min_distance, None
def face_detection(img):
    h, w, _ = img.shape
    img_resize = cv2.resize(img, (320, 240))
    img_resize = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)
    img_resize = img_resize - 127.0
    img_resize = img_resize / 128.0
    results = model.predict(np.expand_dims(img_resize, axis=0))  # result=[background,face,x1,y1,x2,y2]
    output = []
    for result in results:
        left = int(result[2] * w)
        top = int(result[3] * h)
        right = int(result[4] * w)
        bottom = int(result[5] * h)
        output.append([top, right, bottom, left])
    return output

# Load User data

In [7]:
data = load_data()
keys = list(data.keys())

NameError: name 'face_recognition' is not defined

In [ ]:
test_image = face_recognition.load_image_file("1.jpg")

import time
# tik = time.time()
face_location = face_detection(test_image)[0]
top, right, bottom, left = face_location
face = test_image[top:bottom, left:right]
tik = time.time()
min_distance, user_matched = user_verify(face)
tok = time.time()
print(tok -tik)
print(user_matched)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
# fontScale
fontScale = 0.7
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2
video_link = "1.mp4"
cap = cv2.VideoCapture(video_link)
stt = 0
t1 = time.time()
while True:
    t2 = time.time()
    fps = 1/(t2-t1)
    t1 = t2
    stt+=1
    ret, frame = cap.read()
#     frame = cv2.resize(frame, (700, 400))
    if not ret:
        break
    t3 = time.time()
    face_locations = face_detection(frame)
    t4 = time.time()
    N = len(face_locations)
    if N>0:
        for face_location in face_locations:
            top, right, bottom, left = face_location
            face = frame[top:bottom, left:right][:, :, ::-1]
            t5 = time.time()
            min_distance, user_matched = user_verify(face)
            t6 = time.time()
            user, rect_color = [user_matched['name'], (0, 255, 0)] if user_matched is not None else ["Unknown", (255, 0, 0)]
            if min_distance>0.6:
                rect_color = (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), rect_color, 2)
            cv2.putText(frame, user +"("+str(round(min_distance, 2))+")", (left, top -10), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, "FPS: "+str(round(fps)), (30, 30), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, "FPS2: "+str(round(t4-t3, 4)), (30, 60), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, "FPS3: "+str(round(t6-t5, 4)), (30, 90), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow("frame", frame)
        k = cv2.waitKey(1)
        if k == ord('q'):
            cv2.destroyAllWindows()
            break

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())